#Import the necessary libraries

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from efficient_apriori import apriori
import warnings


In [ ]:
#Import the data and clean the dataframes

# Load data 
df_sales = pd.read_excel("Your path here")
df_Item = pd.read_excel("Your path here")
df_Customers = pd.read_excel("Your path here")

# Clean dataframes 
Dataframes = [df_Item, df_Customers, df_sales]
for idx, df in enumerate(Dataframes):
    columns_before = df.columns.tolist()
    df.dropna(axis=1, how="all", inplace=True)

    # Track removed columns
    cols_after = df.columns.to_list()
    removed_cols = set(columns_before) - set(cols_after)
    # Print results
    print(f"DataFrame {idx}: Removed columns with all missing values: {removed_cols or 'None'}")
    print(f"Remaining columns: {cols_after}\n")

# Merge dataframes
df_sales['SoldToCustomer_CD'] = df_sales['SoldToCustomer_CD'].astype(object)
merged_d = df_sales.merge(df_Customers, left_on="SoldToCustomer_CD", right_on="Customer_CD", how="left")
merged_df = merged_d.merge(df_Item, how="left", on=["Item_CD"])

# Columns to keep
columns_to_keep = [
    'SalesDocumentNumber',
    "BrandL1_TX",
    'Date_CD',
    'Item_CD',
    'SoldToCustomer_CD',
    'NetSalesValue',
    'Item_TX',
    'CategoryL1_TX',
    'ItemStatus_CD',
    'Customer_TX',
    'ParentCustomer_CD',
    'ParentCustomer_TX',
    'CustomerGroup_CD',
    'CustomerGroup_TX',
    'CustomerGroupL1_CD',
    'CustomerGroupL1_TX',
    "SalesQuantity",
    "Customer_CD"
]

# Keep specified columns
merged_df = merged_df[columns_to_keep]

# Remove rows with missing values 
merged_df.dropna(how='all', inplace=True)

# Date_CD column to datetime format
merged_df['Date_CD'] = pd.to_datetime(merged_df['Date_CD'], errors='coerce')

# Filter the DataFrame 
merged_df = merged_df[columns_to_keep]

# Remove rows where SalesQuantity is negative
merged_df = merged_df[merged_df['SalesQuantity'] >= 0]

# Calculate Net Sales per Kilo
merged_df['Net Sales per Kilo'] = merged_df.apply(
    lambda row: row['NetSalesValue'] / row['SalesQuantity']
    if pd.notnull(row['NetSalesValue']) and row['SalesQuantity'] > 0 else None,
    axis=1
)

##Defining frecuency

invoice_freq = merged_df.groupby('CustomerGroupL1_CD')['SalesDocumentNumber'].nunique().reset_index()
invoice_freq.columns = ['CustomerGroupL1_CD', 'Invoice_Frequency']

merged_df['YearMonth'] = merged_df['Date_CD'].dt.to_period('M')
monthly_orders = merged_df.groupby(['CustomerGroupL1_CD', 'YearMonth'])['SalesDocumentNumber'].nunique().reset_index()

# Now average monthly frequency
monthly_freq = monthly_orders.groupby('CustomerGroupL1_CD')['SalesDocumentNumber'].mean().reset_index()
monthly_freq.columns = ['CustomerGroupL1_CD', 'Avg_Monthly_Orders']

# Create new invoice key
merged_df['InvoiceKey'] = (
    merged_df['CustomerGroupL1_CD'].astype(str) + "_" + merged_df['Date_CD'].dt.strftime("%Y-%m-%d")
)

# Preview how many unique transactions this defines
print("Unique transactions (InvoiceKey):", merged_df['InvoiceKey'].nunique())

raw_txn_count = (
    merged_df.groupby(["SoldToCustomer_CD", "Date_CD"])["SalesDocumentNumber"]
    .nunique()
    .reset_index(name="Raw_Transaction_Count")
)

print(f"Total raw transactions (store+day level): {len(raw_txn_count)}")


merged_df['InvoiceKey'] = (
    merged_df['CustomerGroupL1_CD'].astype(str) + "_" +
    merged_df['Date_CD'].dt.strftime("%Y-%m-%d")
)

# Counting how many unique InvoiceKeys exist
unique_invoicekey_count = merged_df['InvoiceKey'].nunique()

print(f" Total aggregated transactions (group+day level): {unique_invoicekey_count}")

comparison = pd.DataFrame({
    'Metric': ['Number of Invoices', 'Aggregated invoices by day'],
    'Transaction_Count': [len(raw_txn_count), unique_invoicekey_count]
})

print(comparison)

import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.bar(comparison['Metric'], comparison['Transaction_Count'], color=['skyblue', 'orange'])
plt.title("Comparison of Transaction Definitions")
plt.ylabel("Number of Transactions")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


columns_to_keep = [
    'SalesDocumentNumber',
    "BrandL1_TX",
    'Date_CD',
    'Item_CD',
    'SoldToCustomer_CD',
    'NetSalesValue',
    'Item_TX',
    'CategoryL1_TX',
    'ItemStatus_CD',
    'Customer_TX',
    'ParentCustomer_CD',
    'ParentCustomer_TX',
    'CustomerGroup_CD',
    'CustomerGroup_TX',
    'CustomerGroupL1_CD',
    'CustomerGroupL1_TX',
    "InvoiceKey",
    "SalesQuantity",
    "Customer_CD"
]


In [ ]:
#Contextual data cleaning (Optional)

####Deleting transactions with only one item

# Counting items per invoice
invoicekey_item_counts = merged_df.groupby("InvoiceKey")['Item_CD'].nunique().reset_index()
invoicekey_item_counts.columns = ["InvoiceKey", 'ItemCount']

# Filtering InvoiceKeys with more than one item
valid_invoicekeys = invoicekey_item_counts[invoicekey_item_counts['ItemCount'] > 1]['InvoiceKey']

# Filtering merged_df to only keep those valid transactions
merged_df = merged_df[merged_df['InvoiceKey'].isin(valid_invoicekeys)].copy()

print(f" Transactions dropped: {merged_df['InvoiceKey'].nunique()} kept from original set.")


original_day_txns = merged_df['InvoiceKey'].nunique()
print(f" Original day-level transactions: {original_day_txns}")

   
invoicekey_item_counts = merged_df.groupby('InvoiceKey')['Item_CD'].nunique().reset_index()
invoicekey_item_counts.columns = ['InvoiceKey', 'ItemCount']


valid_invoicekeys = invoicekey_item_counts[invoicekey_item_counts['ItemCount'] > 1]['InvoiceKey']
merged_df_filtered = merged_df[merged_df['InvoiceKey'].isin(valid_invoicekeys)].copy()


filtered_day_txns = merged_df_filtered['InvoiceKey'].nunique()
print(f" Kept day-level transactions after dropping 1-item baskets: {filtered_day_txns}")


dropped = original_day_txns - filtered_day_txns
print(f" Dropped day-level transactions with only 1 item: {dropped}")


merged_df = merged_df[columns_to_keep]

# Remove rows with missing values in the specified columns
merged_df.dropna(how='all', inplace=True)


merged_df = merged_df[columns_to_keep]

# Remove rows where SalesQuantity is negative
merged_df = merged_df[merged_df['SalesQuantity'] >= 0]

# Calculate Net Sales per Kilo

merged_df['Net Sales per Kilo'] = merged_df.apply(
    lambda row: row['NetSalesValue'] / row['SalesQuantity']
    if pd.notnull(row['NetSalesValue']) and row['SalesQuantity'] > 0 else None,
    axis=1
)

##Defining frecuency

invoice_freq = merged_df.groupby('CustomerGroupL1_CD')['SalesDocumentNumber'].nunique().reset_index()
invoice_freq.columns = ['CustomerGroupL1_CD', 'Invoice_Frequency']

merged_df['YearMonth'] = merged_df['Date_CD'].dt.to_period('M')
monthly_orders = merged_df.groupby(['CustomerGroupL1_CD', 'YearMonth'])['SalesDocumentNumber'].nunique().reset_index()


monthly_freq = monthly_orders.groupby('CustomerGroupL1_CD')['SalesDocumentNumber'].mean().reset_index()
monthly_freq.columns = ['CustomerGroupL1_CD', 'Avg_Monthly_Orders']



####Deleting transactions with only one item



invoice_item_counts = merged_df.groupby('InvoiceKey')['Item_CD'].nunique().reset_index()
invoice_item_counts.columns = ['InvoiceKey', 'ItemCount']


one_item_invoices = invoice_item_counts[invoice_item_counts['ItemCount'] == 1]
print(f"Number of invoices with only one item: {len(one_item_invoices)}")


unwanted_brands = ['Category 1', 'Category 2', 'Category 3']
merged_df = merged_df[~merged_df['BrandL1_TX'].isin(unwanted_brands)]
merged_df = merged_df[merged_df['BrandL1_TX'].notna()]


In [ ]:
#Pickle save and load

# Export merged_df as a pickle file
merged_df.to_pickle("Your path here")


### Load the pickle file
merged_df = pd.read_pickle("Your path here")


In [ ]:
#Data analysis and visualization

merged_df['Date_CD'] = pd.to_datetime(merged_df['Date_CD'], errors='coerce')


merged_df['YearMonth'] = merged_df['Date_CD'].dt.to_period('M')


target_brands = ['brand 3', 'brand 2','brand 1']
brand_data = merged_df[merged_df['BrandL1_TX'].isin(target_brands)].copy()


monthly_sales = brand_data.groupby(['BrandL1_TX', 'YearMonth'])['NetSalesValue'].sum().reset_index()


monthly_sales['YearMonth'] = monthly_sales['YearMonth'].dt.to_timestamp()


merged_df['Date_CD'] = pd.to_datetime(merged_df['Date_CD'], errors='coerce')
merged_df['YearMonth'] = merged_df['Date_CD'].dt.to_period('M')

# Filter for 2024
df_2024 = merged_df[merged_df['Date_CD'].dt.year == 2024].copy()


monthly_2024 = (
    df_2024
    .groupby(['Item_TX', 'YearMonth','BrandL1_TX'])['NetSalesValue']
    .sum()
    .reset_index()
)


monthly_2024['YearMonth'] = monthly_2024['YearMonth'].dt.to_timestamp()



monthly_2024_jd = monthly_2024[monthly_2024['BrandL1_TX'] == 'Brand name'].copy()


pivot_2024 = monthly_2024_jd.pivot_table(
    index='YearMonth',
    columns='Item_TX',
    values='NetSalesValue',
    fill_value=0
)




import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=monthly_sales, x='YearMonth', y='NetSalesValue', hue='BrandL1_TX', marker='o')
plt.title(' Monthly Net Sales: brand 1 vs. brand 2 vs. brand 3')
plt.xlabel('Month')
plt.ylabel('Net Sales Value (€)')
plt.grid(True)
plt.tight_layout()
plt.show()


pivot_2024 = monthly_2024.pivot_table(
    index='YearMonth',
    columns='Item_TX',
    values='NetSalesValue',
    fill_value=0
)


start_month = pivot_2024.index.min()
end_month = pivot_2024.index.max()

growth_rates = ((pivot_2024.loc[end_month] - pivot_2024.loc[start_month]) /
                pivot_2024.loc[start_month].replace(0, 1)) * 100


top_growth_items = growth_rates.sort_values(ascending=False).head(5).index.tolist()

print(" Top Growing Products in 2024:")
for item in top_growth_items:
    print(f"- {item}: {growth_rates[item]:.2f}% growth")



top_growth_sales = monthly_2024[monthly_2024['Item_TX'].isin(top_growth_items)]


plt.figure(figsize=(14, 6))
sns.lineplot(data=top_growth_sales, x='YearMonth', y='NetSalesValue', hue='Item_TX', marker='o')
plt.title("Sales of Top 5 Growing Products (2024)")
plt.xlabel("Month")
plt.ylabel("Net Sales (€)")
plt.grid(True)
plt.tight_layout()
plt.show()



merged_df['Date_CD'] = pd.to_datetime(merged_df['Date_CD'], errors='coerce')
merged_df['YearMonth'] = merged_df['Date_CD'].dt.to_period('M')

# Filter for 2024
df_2024 = merged_df[merged_df['Date_CD'].dt.year == 2024].copy()

# Group monthly sales by item
monthly_2024 = (
    df_2024
    .groupby(['Item_TX', 'YearMonth'])['NetSalesValue']
    .sum()
    .reset_index()
)
monthly_2024['YearMonth'] = monthly_2024['YearMonth'].dt.to_timestamp()

# Pivot: rows = months, columns = items
pivot_2024 = monthly_2024.pivot_table(
    index='YearMonth',
    columns='Item_TX',
    values='NetSalesValue',
    fill_value=0
)

# Calculate growth from first to last month
start_month = pivot_2024.index.min()
end_month = pivot_2024.index.max()

growth_rates = ((pivot_2024.loc[end_month] - pivot_2024.loc[start_month]) /
                pivot_2024.loc[start_month].replace(0, 1)) * 100

# Top 5 growing items
top_growth_items = growth_rates.sort_values(ascending=False).head(5).index.tolist()

print(" Top Growing Products in 2024:")
for item in top_growth_items:
    print(f"- {item}: {growth_rates[item]:.2f}% growth")

# Plot top growing products
top_growth_sales = monthly_2024[monthly_2024['Item_TX'].isin(top_growth_items)]



plt.figure(figsize=(14, 6))
sns.lineplot(data=top_growth_sales, x='YearMonth', y='NetSalesValue', hue='Item_TX', marker='o')
plt.title(" Sales of Top 5 Growing Products (2024)")
plt.xlabel("Month")
plt.ylabel("Net Sales (€)")
plt.grid(True)
plt.tight_layout()
plt.show()





In [ ]:
#Additional features for clustering and normalizing

# 1. Average Days Between Orders per Customer
# Sort invoices by customer and date, then calculate time differences
merged_df_sorted = merged_df.sort_values(by=['CustomerGroupL1_CD', 'Date_CD'])
merged_df_sorted['Days_Since_Last_Order'] = merged_df_sorted.groupby('CustomerGroupL1_CD')['Date_CD'].diff().dt.days

# Now calculate average days between orders
avg_days_between_orders = merged_df_sorted.groupby('CustomerGroupL1_CD')['Days_Since_Last_Order'].mean().reset_index()
avg_days_between_orders.columns = ['CustomerGroupL1_CD', 'Avg_Days_Between_Orders']





# NetSalesValue per customer per brand
brand_spend = merged_df.groupby(['CustomerGroupL1_CD', 'BrandL1_TX'])['NetSalesValue'].sum().reset_index()

#  NetSalesValue per customer
total_spend = merged_df.groupby('CustomerGroupL1_CD')['NetSalesValue'].sum().reset_index()
total_spend.columns = ['CustomerGroupL1_CD', 'Total_NetSales']

# Spend per brand
brand_spend = brand_spend.merge(total_spend, on='CustomerGroupL1_CD')
brand_spend['Brand_Spend_Percent'] = brand_spend['NetSalesValue'] / brand_spend['Total_NetSales']

brand_spend_matrix = brand_spend.pivot_table(
    index='CustomerGroupL1_CD',
    columns='BrandL1_TX',
    values='Brand_Spend_Percent',
    fill_value=0  
).reset_index()


brand_spend_matrix.columns = ['CustomerGroupL1_CD'] + [f'Brand_Percent_{col}' for col in brand_spend_matrix.columns if col != 'CustomerGroupL1_CD']




#  Calculate invoice frequency per customer (number of unique invoice dates)
invoice_freq_all = merged_df.groupby('CustomerGroupL1_CD')['Date_CD'].nunique().rename("All_Time_Freq")
invoice_freq_2024 = df_2024.groupby('CustomerGroupL1_CD')['Date_CD'].nunique().rename("Freq_2024")


freq_df = pd.concat([invoice_freq_all, invoice_freq_2024], axis=1).fillna(0)

#  Plot histogram of both frequencies
plt.figure(figsize=(10, 5))
sns.histplot(freq_df["All_Time_Freq"], bins=20, color='skyblue', label='All Time', kde=False)
sns.histplot(freq_df["Freq_2024"], bins=20, color='salmon', label='2024', kde=False)

plt.title(" Invoice Frequency Distribution: All Time vs. 2024")
plt.xlabel("Number of Invoices per Customer")
plt.ylabel("Customer Count")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



# Aggregate metrics per customer
customer_summary = merged_df.groupby('CustomerGroupL1_CD').agg(
    Total_Kilos=('SalesQuantity', 'sum'),
    Total_NetSales=('NetSalesValue', 'sum'),
    Invoice_Frequency=('InvoiceKey', pd.Series.nunique)
).reset_index()

# Calculate Net Sales per Kilo
customer_summary['Avg_NetSalesPerKilo'] = customer_summary['Total_NetSales'] / customer_summary['Total_Kilos']

customer_summary = customer_summary.merge(avg_days_between_orders, on='CustomerGroupL1_CD', how='left')
customer_summary = customer_summary.merge(brand_spend_matrix, on='CustomerGroupL1_CD', how='left')


import numpy as np



#Capping the clustering features at the 90 percentile
features = ['Total_Kilos', 'Total_NetSales', 'Avg_NetSalesPerKilo', 'Invoice_Frequency']

customer_summary_capped = customer_summary.copy()


for feature in features:
    cap_value = customer_summary_capped[feature].quantile(0.90)
    customer_summary_capped[feature] = np.where(
        customer_summary_capped[feature] > cap_value,
        cap_value,
        customer_summary_capped[feature]
    )



import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(data=customer_summary_capped[features])
plt.title("Boxplot After Winsorization (99th Percentile Cap)")
plt.xticks(rotation=45)
plt.show()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


# Use capped version
valid_rows = customer_summary_capped[features].replace([np.inf, -np.inf], np.nan).dropna()
valid_data = valid_rows.copy()
valid_ids = customer_summary_capped.loc[valid_rows.index, 'CustomerGroupL1_CD'].reset_index(drop=True)


normalized = scaler.fit_transform(valid_data) 
###Prefered a DataFrame for better readability instead of a numpy array

normalized_df = pd.DataFrame(
    scaler.fit_transform(valid_data),
    columns=valid_data.columns
)

customer_cluster_input = pd.concat([customer_summary_capped[["CustomerGroupL1_CD"]],normalized_df], axis=1)

import seaborn as sns
import matplotlib.pyplot as plt

plot_df = customer_cluster_input.copy()

#create pairplot of the normalized data

sns.pairplot(plot_df.drop(columns=['CustomerGroupL1_CD']))
plt.suptitle("Pairwise Plot of Normalized Customer Features", y=1.02)
plt.show()

Clustering stage: Grouping the different users into differentiated groups

In [ ]:
###Defining best K for KMeans
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans_test = KMeans(n_clusters=k, random_state=42)
    labels_k = kmeans_test.fit_predict(normalized_df)
    inertias.append(kmeans_test.inertia_)
    silhouette_scores.append(silhouette_score(normalized_df, labels_k))

# Plot results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(k_range, inertias, marker='o')
plt.title("Elbow Method")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")

plt.subplot(1, 2, 2)
plt.plot(k_range, silhouette_scores, marker='o')
plt.title("Silhouette Scores")
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")

plt.tight_layout()
plt.show()

# Exact scores
for k, inertia, sil in zip(k_range, inertias, silhouette_scores):
    print(f"k={k}: Inertia={inertia:.2f}, Silhouette Score={sil:.4f}")


kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(normalized_df)

clustered_customers = pd.DataFrame({
    'CustomerGroupL1_CD': valid_ids,
    'Cluster': labels
})


from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(normalized_df)
    inertias.append(kmeans.inertia_)



from sklearn.metrics import silhouette_score

scores = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(normalized_df)
    score = silhouette_score(normalized_df, labels)
    scores.append(score)



from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

inertias = []
silhouettes = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(normalized_df)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(normalized_df, labels))



best_k = 3
kmeans_final = KMeans(n_clusters=best_k, random_state=42)
final_labels = kmeans_final.fit_predict(normalized_df)

# Assign cluster labels back to customers
clustered_customers = pd.DataFrame({
    'CustomerGroupL1_CD': valid_ids,
    'Cluster': final_labels
})

# Merge clusters into customer_summary
customer_summary_clustered_capped = customer_summary_capped.merge(clustered_customers, on='CustomerGroupL1_CD', how='inner')



print(customer_summary_clustered_capped['Cluster'].value_counts())




Graphic Visualization of clusters and their centroids

In [ ]:
# PCA
pca_features = normalized_df  # Final version
pca = PCA(n_components=2)
pca_result = pca.fit_transform(pca_features)

# Create a plotting DataFrame
pca_df = pd.DataFrame({
    'PCA1': pca_result[:, 0],
    'PCA2': pca_result[:, 1],
    'Cluster': clustered_customers['Cluster'].values  #Final clusters
})

# Plot
plt.figure(figsize=(10, 7))
sns.scatterplot(data=pca_df, x='PCA1', y='PCA2', hue='Cluster', palette='tab10', s=60)
plt.title(' PCA: Customer Clusters Visualized in 2D')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.grid(True)
plt.tight_layout()
plt.show()


plot_df = customer_summary_clustered_capped[['Cluster'] + features].copy()

melted = plot_df.melt(id_vars='Cluster', var_name='Feature', value_name='Value')

# Plot
plt.figure(figsize=(14, 8))
sns.boxplot(data=melted, x='Feature', y='Value', hue='Cluster')
plt.xticks(rotation=45)
plt.title('Feature Distributions by Cluster')
plt.grid(True)
plt.tight_layout()
plt.show()


normalized_plot_df = normalized_df.copy()
normalized_plot_df['Cluster'] = clustered_customers['Cluster'].values



melted_norm = normalized_plot_df.melt(id_vars='Cluster', var_name='Feature', value_name='Normalized Value')

# Plot
plt.figure(figsize=(14, 8))
sns.boxplot(data=melted_norm, x='Feature', y='Normalized Value', hue='Cluster')
plt.title(' Normalized Feature Distribution by Cluster')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()



# How many customers per cluster
print(customer_summary_clustered_capped['Cluster'].value_counts())


customer_summary_clustered_capped['Cluster'].value_counts().plot(kind='bar')
plt.title('🧮 Customers per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Number of Customers')
plt.grid(True)
plt.show()


####Analytics

# Select relevant columns
metrics = ['Total_Kilos', 'Total_NetSales', 'Avg_NetSalesPerKilo', 'Invoice_Frequency']

# Overall summary statistics
print(" General Summary Statistics (All Customers):")
general_stats = customer_summary_clustered_capped[metrics].describe().T.round(2)
print(general_stats)

# %%
# Summary statistics by cluster
print("\n Summary Statistics by Cluster:")
cluster_stats = customer_summary_clustered_capped.groupby('Cluster')[metrics].describe().round(2)
print(cluster_stats)


cluster_stats_flat = cluster_stats.stack().unstack(1).round(2)
#cluster_stats_flat.to_excel("cluster_metrics_summary.xlsx")

# %%
import matplotlib.pyplot as plt

# Metrics you want to plot
metrics = ['Total_Kilos', 'Total_NetSales', 'Avg_NetSalesPerKilo', 'Invoice_Frequency']

# Group by cluster
cluster_means = customer_summary_clustered_capped.groupby('Cluster')[metrics].mean()
cluster_stds = customer_summary_clustered_capped.groupby('Cluster')[metrics].std()

# Plot each metric
for metric in metrics:
    plt.figure(figsize=(6, 4))
    y = cluster_means[metric]
    err = cluster_stds[metric]

    # Plot with error bars (std dev)
    plt.bar(y.index.astype(str), y.values, yerr=err.values, capsize=5, color='skyblue')
    plt.title(f" {metric} by Cluster")
    plt.xlabel("Cluster")
    plt.ylabel(metric)
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.show()



# List of variables to plot
metrics = ['Total_Kilos', 'Total_NetSales', 'Avg_NetSalesPerKilo', 'Invoice_Frequency']

# One boxplot per metric
for metric in metrics:
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=customer_summary_clustered_capped, x='Cluster', y=metric, palette='Set2')
    plt.title(f" Distribution of {metric} by Cluster")
    plt.xlabel("Cluster")
    plt.ylabel(metric)
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.show()


brand_cols = [col for col in customer_summary_clustered_capped.columns if col.startswith("Brand_Percent_")]

melted_brand_spend = customer_summary_clustered_capped.melt(
    id_vars='Cluster',
    value_vars=brand_cols,
    var_name='Brand',
    value_name='Percent_Spent'
)

# Average spend per brand per cluster
plt.figure(figsize=(14, 6))
sns.barplot(data=melted_brand_spend, x='Brand', y='Percent_Spent', hue='Cluster')
plt.title(" Average Brand Spend % by Cluster")
plt.xticks(rotation=45)
plt.ylabel("Average % of Spend")
plt.grid(True)
plt.tight_layout()
plt.show()

# PCA for feature contributions
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd


# Run PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(normalized_df)

# Create a DataFrame with feature loadings
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=normalized_df.columns
)

# Plot feature contributions
ax = loadings.plot(kind='barh', figsize=(10, 6), title=' Feature Contributions to Principal Components')
plt.axvline(0, color='black', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

# Display top contributing features for each PC
print("\n Top Contributors to PC1:")
print(loadings['PC1'].abs().sort_values(ascending=False).head(5))

print("\n Top Contributors to PC2:")
print(loadings['PC2'].abs().sort_values(ascending=False).head(5))




In [ ]:
# Transactions labeled with clusters:
merged_df_clustered = merged_df.merge(
    customer_summary_clustered_capped[['CustomerGroupL1_CD','Cluster']],
    on='CustomerGroupL1_CD',
    how='inner'
)
merged_df_clustered['Cluster'] = merged_df_clustered['Cluster'].astype(int)



In [ ]:


#Defining the most relevant associations between propducts 

print("DataFrame shape:", clustered_df.shape)
print("Columns:", clustered_df.columns.tolist())


print(clustered_df.head(10).to_string(index=False))


for cid in sorted(clustered_df['cluster'].unique()):
    print(f"\n--- Cluster {cid} ---")
    sub = clustered_df[clustered_df['cluster']==cid][
        ['antecedent','consequent','cust_bought_ant','cust_diff_count']
    ]
# rule + counts
    print(sub.to_string(index=False))

summary = clustered_df[['cluster','antecedent','consequent','cust_bought_ant','cust_diff_count']]
print("\nAll clusters, all rules:\n", summary.to_string(index=False))



Use the Association rule Library Apriori - Rule mining

In [ ]:

# All transactions
df_all = merged_df_clustered.copy()
#BY INVOICE
transactions = df_all.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()

#Apriori over entire dataset
itemsets, rules = apriori(
    transactions,
    min_support=0.05,
    min_confidence=0.5
)

def extract_counts(rule, df):
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(df[df['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(df[df['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff      = cust_ant - cust_cons
    return cust_ant, cust_cons, diff

# Prepare three sections: all, top15, next15
sorted_rules = sorted(rules, key=lambda r: r.support, reverse=True)
sections = {
    'All_by_Support' : sorted_rules,
    'Top15'          : sorted_rules[:15],
    'Next15'         : sorted_rules[15:30],
}

# Build DataFrames
frames = {}
for name, sel in sections.items():
    recs = []
    for rule in sel:
        cust_ant, cust_cons, diff = extract_counts(rule, df_all)
        recs.append({
            'antecedent'      : ', '.join(rule.lhs),
            'consequent'      : ', '.join(rule.rhs),
            'support'         : rule.support,
            'confidence'      : rule.confidence,
            'lift'            : rule.lift,
            'cust_bought_ant' : len(cust_ant),
            'cust_diff_count' : len(diff),
            'cust_diff_list'  : list(diff)
        })
    frames[name] = pd.DataFrame(recs)

# Id´s -Names
cust_map = (
    df_all[['ParentCustomer_CD','Customer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['Customer_TX']
    .to_dict()
)

for df in frames.values():
    # clean IDs
    df['Missing_ParentCustomer_CDs'] = df['cust_diff_list'].apply(
        lambda lst: [int(x) for x in lst if pd.notnull(x)]
    )
    # map to names
    df['Missing_ParentCustomer_TXs'] = df['Missing_ParentCustomer_CDs'].apply(
        lambda ids: "; ".join(cust_map.get(cid, f"ID:{cid}") for cid in ids)
    )

# Export 
out_path = "G:/Mi unidad/Antwerp/Data engineering/Solved/Market basket/mba_general_rules_by_support.xlsx"
with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
    frames['All_by_Support'].to_excel(writer, sheet_name='General_All_by_Support', index=False)
    frames['Top15'].to_excel(writer,      sheet_name='General_Top15',          index=False)
    frames['Next15'].to_excel(writer,     sheet_name='General_Next15',         index=False)
print(" General MBA rules exported to:", out_path)

# Plotting
plt.figure(figsize=(8,6), tight_layout=True)
sns.barplot(
    data=frames['Top15'],
    y='antecedent',
    x='cust_diff_count',
    hue='consequent',
    dodge=False
)
plt.title(" Overall: # Customers Bought Antecedent but NOT Consequent\n(Top 15 Rules by Support)")
plt.xlabel("Number of Customers Missing Consequent")
plt.ylabel("Antecedent")
plt.legend(title="Consequent", bbox_to_anchor=(1.05,1), loc='upper left')
plt.show()

##Counting Nan Values


In [ ]:
############  Clustered Anlysis

#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from efficient_apriori import apriori


###Cleared transactions general

# General transactions and run Apriori 
transactions = merged_df_clustered.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()
itemsets, rules = apriori(transactions, min_support=0.05, min_confidence=0.0)


def clean_ids(id_list):
    return [int(x) for x in id_list if pd.notnull(x)]

#  Top-15 by support 
general_records = []
for rule in sorted(rules, key=lambda r: r.support, reverse=True)[:15]:
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff = list(cust_ant - cust_cons)

    general_records.append({
        'antecedent'      : ', '.join(rule.lhs),
        'consequent'      : ', '.join(rule.rhs),
        'support'         : rule.support,
        'confidence'      : rule.confidence,
        'lift'            : rule.lift,
        'cust_bought_ant' : len(cust_ant),
        'raw_diff_list'   : diff,
        'clean_diff_list' : clean_ids(diff),
        'diff_count'      : len(clean_ids(diff))
    })

general_top15 = pd.DataFrame(general_records)

#  Next-15 by support 
general_records_2 = []
for rule in sorted(rules, key=lambda r: r.support, reverse=True)[15:30]:
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff = list(cust_ant - cust_cons)

    general_records_2.append({
        'antecedent'      : ', '.join(rule.lhs),
        'consequent'      : ', '.join(rule.rhs),
        'support'         : rule.support,
        'confidence'      : rule.confidence,
        'lift'            : rule.lift,
        'cust_bought_ant' : len(cust_ant),
        'raw_diff_list'   : diff,
        'clean_diff_list' : clean_ids(diff),
        'diff_count'      : len(clean_ids(diff))
    })

general_next15 = pd.DataFrame(general_records_2)

# Build ParentCustomer_CD - ParentCustomer_TX 
cust_map = (
    merged_df_clustered[['ParentCustomer_CD','ParentCustomer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['ParentCustomer_TX']
    .to_dict()
)

# IDs  -names 
for df in (general_top15, general_next15):
    df['Missing_ParentCustomer_TXs'] = df['clean_diff_list'].apply(
        lambda ids: [cust_map.get(i, f"ID:{i}") for i in ids]
    )
    df['Missing_ParentCustomer_TXs_str'] = df['Missing_ParentCustomer_TXs'].apply(lambda L: "; ".join(L))

# Export
out_path = "Your path here"
with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
    general_top15 .to_excel(writer, sheet_name='General_Top15', index=False)
    general_next15.to_excel(writer, sheet_name='General_Next15', index=False)
print("Exported with customer-names to:", out_path)

for df, title in [(general_top15, "Top 15"), (general_next15, "Next 15")]:
    pdf = df[df['diff_count'] > 0]
    plt.figure(figsize=(8, max(4, len(pdf)*0.5)), tight_layout=True)
    sns.barplot(
        data=pdf,
        y='antecedent',
        x='diff_count',
        hue='consequent',
        dodge=False
    )
    plt.title(f" Overall: # Customers Bought Antecedent but NOT Consequent\n({title} Rules by Support)")
    plt.xlabel("Number of Customers Missing Consequent")
    plt.ylabel("Antecedent Item")
    plt.legend(title="Consequent", bbox_to_anchor=(1.05,1), loc='upper left')
    plt.show()


In [ ]:


def export_item_support_table(merged_df_clustered, output_path):


    merged_df_clustered['Cluster'] = merged_df_clustered['Cluster'].astype(int)

    # Total number of unique invoices
    total_txns = merged_df_clustered['InvoiceKey'].nunique()
    total_by_cluster = merged_df_clustered.groupby('Cluster')['InvoiceKey'].nunique()

    # Item support per cluster
    item_cluster_support = (
        merged_df_clustered
        .groupby(['Item_TX', 'Cluster'])['InvoiceKey']
        .nunique()
        .unstack(fill_value=0)
    )

    # Normalize to get support
    for cluster in item_cluster_support.columns:
        item_cluster_support[cluster] = item_cluster_support[cluster] / total_by_cluster[cluster]

    # Add overall support
    overall_support = (
        merged_df_clustered
        .groupby('Item_TX')['InvoiceKey']
        .nunique()
        .div(total_txns)
    )
    item_cluster_support['Overall'] = overall_support

    # Convert to percentage
    item_cluster_support_pct = (item_cluster_support * 100).round(1).astype(str) + '%'

    # Sort by Overall
    item_cluster_support_pct = item_cluster_support_pct.sort_values(by='Overall', ascending=False)

    # Export
    item_cluster_support_pct.to_excel(output_path)
    print(f" Support report exported to: {output_path}")

# ==== Usage ====
output_path = "Your path here"
export_item_support_table(merged_df_clustered, output_path)


General Association rules

In [ ]:
# Cluster‐specific Apriori to get rule lists
from efficient_apriori import apriori

cluster_rules_dict = {}
for cid in sorted(merged_df_clustered['Cluster'].unique()):
    dfc = merged_df_clustered[merged_df_clustered['Cluster']==cid]
    txns = dfc.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()
    if txns:
        itemsets, rules_list = apriori(txns, min_support=0.06, min_confidence=0.5)
    else:
        rules_list = []
    cluster_rules_dict[cid] = rules_list

# Top-15‐by‐support DataFrame per cluster
cluster_rows = []
for cid, rules_list in cluster_rules_dict.items():
    # take the 15 highest‐support rules
    for rule in sorted(rules_list, key=lambda r: r.support, reverse=True)[:15]:
        ant, cons = set(rule.lhs), set(rule.rhs)
        dfc = merged_df_clustered[merged_df_clustered['Cluster']==cid]
        cust_ant  = set(dfc[dfc['Item_TX'].isin(ant)]['ParentCustomer_CD'])
        cust_cons = set(dfc[dfc['Item_TX'].isin(cons)]['ParentCustomer_CD'])
        diff = cust_ant - cust_cons
        
        cluster_rows.append({
            'cluster':          cid,
            'antecedent':       ', '.join(rule.lhs),
            'consequent':       ', '.join(rule.rhs),
            'support':          rule.support,
            'confidence':       rule.confidence,
            'lift':             rule.lift,
            'cust_bought_ant':  len(cust_ant),
            'cust_diff_count':  len(diff),
            'cust_diff_list':   list(diff)
        })

clustered_df = pd.DataFrame(cluster_rows)

# 4) Export  the clustered sheet 
path = "Your path here"
with pd.ExcelWriter(path, engine='openpyxl') as writer:
    clustered_df.to_excel(writer, sheet_name='ByCluster_Top15_by_Support', index=False)

print(" Clustered top-15 rules exported to:", path)


In [ ]:

print("DataFrame shape:", clustered_df.shape)
print("Columns:", clustered_df.columns.tolist())


print(clustered_df.head(10).to_string(index=False))


for cid in sorted(clustered_df['cluster'].unique()):
    print(f"\n--- Cluster {cid} ---")
    sub = clustered_df[clustered_df['cluster']==cid][
        ['antecedent','consequent','cust_bought_ant','cust_diff_count']
    ]
# rule + counts
    print(sub.to_string(index=False))

summary = clustered_df[['cluster','antecedent','consequent','cust_bought_ant','cust_diff_count']]
print("\nAll clusters, all rules:\n", summary.to_string(index=False))


In [ ]:
###Clustered Cleaned count ()



def count_valid(ids):

    return sum(1 for x in ids if pd.notnull(x))

clustered_df['cust_diff_count_clean'] = clustered_df['cust_diff_list'].apply(count_valid)

# Drop any rules where that cleaned count is zero
plot_df = clustered_df[clustered_df['cust_diff_count_clean'] > 0].copy()


for cid in sorted(plot_df['cluster'].unique()):
    dfc = plot_df[plot_df['cluster'] == cid]

    plt.figure(figsize=(8, max(4, len(dfc)*0.5)))
    sns.barplot(
        data=dfc,
        y='antecedent',
        x='cust_diff_count_clean',       # use the cleaned count
        hue='consequent',
        dodge=False
    )
    plt.title(f"Cluster {cid}: # Customers Bought Antecedent but NOT Consequent")
    plt.xlabel("Number of Customers Missing Consequent")
    plt.ylabel("Antecedent Item")
    plt.legend(title="Consequent", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


In [ ]:
#Finding business opportunities with the customers who bought only one of the two products relationship

In [ ]:


#  list of rules for each cluster.
cluster_rules_dict = {}
for cid in sorted(merged_df_clustered['Cluster'].unique()):
    dfc = merged_df_clustered[merged_df_clustered['Cluster'] == cid]
    transactions = dfc.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()
    if transactions:
        _, rules_list = apriori(transactions, min_support=0.06, min_confidence=0.5)
    else:
        rules_list = []
    cluster_rules_dict[cid] = rules_list

# Rules ranked 16–30 by support, per cluster 
cluster_rows = []
for cid, rules_list in cluster_rules_dict.items():
    # sort by support descending
    sorted_rules = sorted(rules_list, key=lambda r: r.support, reverse=True)
    
    # take rules #16 through #30
    for rule in sorted_rules[15:30]:
        ant, cons = set(rule.lhs), set(rule.rhs)
        dfc = merged_df_clustered[merged_df_clustered['Cluster'] == cid]
        
        cust_ant  = set(dfc[dfc['Item_TX'].isin(ant)]['ParentCustomer_CD'])
        cust_cons = set(dfc[dfc['Item_TX'].isin(cons)]['ParentCustomer_CD'])
        diff      = cust_ant - cust_cons
        
        cluster_rows.append({
            'cluster':           cid,
            'antecedent':        ', '.join(rule.lhs),
            'consequent':        ', '.join(rule.rhs),
            'support':           rule.support,
            'confidence':        rule.confidence,
            'lift':              rule.lift,
            'cust_bought_ant':   len(cust_ant),
            'cust_diff_list':    list(diff)
        })

clustered_next15 = pd.DataFrame(cluster_rows)

# Clean out NaNs 
def count_valid(ids):
    return sum(1 for x in ids if pd.notnull(x))

clustered_next15['cust_diff_count'] = (
    clustered_next15['cust_diff_list']
    .apply(count_valid)
)

plot_df = clustered_next15[clustered_next15['cust_diff_count'] > 0].copy()

# ParentCustomer_CD → Customer_TX
cust_map = (
    merged_df_clustered[['ParentCustomer_CD', 'Customer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['Customer_TX']
    .to_dict()
)
def map_names(id_list):
    # filter out any nan
    clean_ids = [cid for cid in id_list if pd.notnull(cid)]
    # map to names, falling back to “ID:123” if missing
    names = [cust_map.get(cid, f"ID:{int(cid)}") for cid in clean_ids]
    return "; ".join(names)

clustered_df['cust_diff_names'] = clustered_df['cust_diff_list'].apply(map_names)

# final export columns
export_cols = [
    'cluster',
    'antecedent',
    'consequent',
    'support',
    'confidence',
    'lift',
    'cust_bought_ant',
    'cust_diff_count',
    'cust_diff_list',
    'cust_diff_names'
]
export_df = clustered_df[export_cols].copy()

path = "G:/Mi unidad/Antwerp/Data engineering/Solved/Market basket/" \
       "mba_clustered_top15_by_support_with_names.xlsx"
with pd.ExcelWriter(path, engine='openpyxl') as writer:
    export_df.to_excel(
        writer,
        sheet_name='ByCluster_Top15_by_Support',
        index=False
    )

print(" Clustered top-15 rules (with cust_diff_names) exported to:", path)



In [ ]:


#Dataframe
cust_map = (
    merged_df_clustered[['ParentCustomer_CD', 'Customer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['Customer_TX']
    .to_dict()
)

def map_names(id_list):
    names = [cust_map.get(i, f"ID:{i}") for i in id_list if pd.notnull(i)]
    return "; ".join(names)

clustered_next15['cust_diff_names'] = (
    clustered_next15['cust_diff_list']
    .apply(map_names)
)

# Select columns 
export_cols = [
    'cluster',
    'antecedent',
    'consequent',
    'support',
    'confidence',
    'lift',
    'cust_bought_ant',
    'cust_diff_count',
    'cust_diff_list',
    'cust_diff_names'
]
export_df = clustered_next15[export_cols].copy()

# Output
output_path = "Your path here"

with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    export_df.to_excel(
        writer,
        sheet_name='ByCluster_Next15_by_Support',
        index=False
    )

print(f" Next-15-by-support clustered rules exported to: {output_path}")


In [ ]:
###Cleaned Clustered rules
### TOP 15 - TOP 30

#Export paths
path_top15   = "Your path here"
path_next15  = "Your path here"


df1 = (
    pd.read_excel(path_top15, sheet_name="ByCluster_Top15_by_Support")
      .assign(rule_set="1-15")
)
df2 = (
    pd.read_excel(path_next15, sheet_name="ByCluster_Next15_by_Support")
      .assign(rule_set="16-30")
)

df = pd.concat([df1, df2], ignore_index=True)

# Clean NaN values
df = df[df['cust_diff_count'].notna()]

#Plotting
for cid in sorted(df['cluster'].unique()):
    for rs in ["1-15","16-30"]:
        sub = df[(df['cluster']==cid) & (df['rule_set']==rs)]
        if sub.empty: 
            continue

        plt.figure(figsize=(10, max(4, len(sub)*0.5)))
        ax = sns.barplot(
            data=sub,
            y="antecedent",
            x="cust_diff_count",
            hue="consequent",
            dodge=False
        )
        ax.set_title(f"Cluster {cid} – Rules {rs} → #Cust bought Ant not Cons")
        ax.set_xlabel("Count of Customers")
        ax.set_ylabel("Antecedent Item")
        ax.legend(title="Consequent", bbox_to_anchor=(1.05,1), loc="upper left")

      
        plt.tight_layout()
        plt.show()


In [ ]:


# Paths 
path_top15  = "Your exporting path"
path_next15 = "Your exporting path"

df1 = (
    pd.read_excel(path_top15, sheet_name="ByCluster_Top15_by_Support")
      .assign(rule_set="1-15")
)
df2 = (
    pd.read_excel(path_next15, sheet_name="ByCluster_Next15_by_Support")
      .assign(rule_set="16-30")
)

combined = pd.concat([df1, df2], ignore_index=True)

#  Clean 
combined['cust_diff_count'] = combined['cust_diff_list'].apply(lambda lst: sum(1 for x in lst if pd.notnull(x)))
combined = combined[combined['cust_diff_count'] > 0].copy()




#  Build DataFrame, preserving both IDs and names:
out = combined[[
    'cluster',
    'rule_set',
    'antecedent',
    'consequent',
    'support',
    'confidence',
    'lift',
    'cust_bought_ant',
    'cust_diff_count',
    'cust_diff_list',    # raw ParentCustomer_CD list
    'cust_diff_names'    # ParentCustomer_TX list
]].rename(columns={
    'cluster'         : 'Cluster',
    'rule_set'        : 'Rule_Set',
    'antecedent'      : 'Antecedent',
    'consequent'      : 'Consequent',
    'support'         : 'Support',
    'confidence'      : 'Confidence',
    'lift'            : 'Lift',
    'cust_bought_ant' : '#Cust_Bought_Ant',
    'cust_diff_count' : '#Cust_Missing_Cons',
    'cust_diff_list'  : 'Missing_ParentCustomer_CDs',
    'cust_diff_names' : 'Missing_ParentCustomer_TXs'
})

# 6) Export: one sheet per cluster 
output_path = "Your path here"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for cid, sub in out.groupby('Cluster'):
        sheet = f"Cluster_{cid}"
        # drop the Cluster column since sheet name encodes it
        sub.drop(columns=['Cluster']).to_excel(writer, sheet_name=sheet, index=False)

print(" Opportunity workbook (with IDs) written to:", output_path)


In original code the clustered MBA

In [ ]:



# All transactions
df_all = merged_df_clustered.copy()
#BY INVOICE
transactions = df_all.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()

#Apriori over entire dataset
itemsets, rules = apriori(
    transactions,
    min_support=0.05,
    min_confidence=0.5
)

def extract_counts(rule, df):
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(df[df['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(df[df['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff      = cust_ant - cust_cons
    return cust_ant, cust_cons, diff

# Prepare three sections: all, top15, next15
sorted_rules = sorted(rules, key=lambda r: r.support, reverse=True)
sections = {
    'All_by_Support' : sorted_rules,
    'Top15'          : sorted_rules[:15],
    'Next15'         : sorted_rules[15:30],
}

# Build DataFrames
frames = {}
for name, sel in sections.items():
    recs = []
    for rule in sel:
        cust_ant, cust_cons, diff = extract_counts(rule, df_all)
        recs.append({
            'antecedent'      : ', '.join(rule.lhs),
            'consequent'      : ', '.join(rule.rhs),
            'support'         : rule.support,
            'confidence'      : rule.confidence,
            'lift'            : rule.lift,
            'cust_bought_ant' : len(cust_ant),
            'cust_diff_count' : len(diff),
            'cust_diff_list'  : list(diff)
        })
    frames[name] = pd.DataFrame(recs)

# Id´s -Names
cust_map = (
    df_all[['ParentCustomer_CD','Customer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['Customer_TX']
    .to_dict()
)

for df in frames.values():
    # clean IDs
    df['Missing_ParentCustomer_CDs'] = df['cust_diff_list'].apply(
        lambda lst: [int(x) for x in lst if pd.notnull(x)]
    )
    # map to names
    df['Missing_ParentCustomer_TXs'] = df['Missing_ParentCustomer_CDs'].apply(
        lambda ids: "; ".join(cust_map.get(cid, f"ID:{cid}") for cid in ids)
    )

# Export 
out_path = "Your exporting path"
with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
    frames['All_by_Support'].to_excel(writer, sheet_name='General_All_by_Support', index=False)
    frames['Top15'].to_excel(writer,      sheet_name='General_Top15',          index=False)
    frames['Next15'].to_excel(writer,     sheet_name='General_Next15',         index=False)
print(" General MBA rules exported to:", out_path)

# Plotting
plt.figure(figsize=(8,6), tight_layout=True)
sns.barplot(
    data=frames['Top15'],
    y='antecedent',
    x='cust_diff_count',
    hue='consequent',
    dodge=False
)
plt.title(" Overall: # Customers Bought Antecedent but NOT Consequent\n(Top 15 Rules by Support)")
plt.xlabel("Number of Customers Missing Consequent")
plt.ylabel("Antecedent")
plt.legend(title="Consequent", bbox_to_anchor=(1.05,1), loc='upper left')
plt.show()

##Counting Nan Values


In [ ]:
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from efficient_apriori import apriori


###Cleared transactions general

# General transactions and run Apriori 
transactions = merged_df_clustered.groupby('InvoiceKey')['Item_TX'].apply(list).tolist()
itemsets, rules = apriori(transactions, min_support=0.05, min_confidence=0.0)


def clean_ids(id_list):
    return [int(x) for x in id_list if pd.notnull(x)]

#  Top-15 by support 
general_records = []
for rule in sorted(rules, key=lambda r: r.support, reverse=True)[:15]:
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff = list(cust_ant - cust_cons)

    general_records.append({
        'antecedent'      : ', '.join(rule.lhs),
        'consequent'      : ', '.join(rule.rhs),
        'support'         : rule.support,
        'confidence'      : rule.confidence,
        'lift'            : rule.lift,
        'cust_bought_ant' : len(cust_ant),
        'raw_diff_list'   : diff,
        'clean_diff_list' : clean_ids(diff),
        'diff_count'      : len(clean_ids(diff))
    })

general_top15 = pd.DataFrame(general_records)

#  Next-15 by support 
general_records_2 = []
for rule in sorted(rules, key=lambda r: r.support, reverse=True)[15:30]:
    ant, cons = set(rule.lhs), set(rule.rhs)
    cust_ant  = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(ant)]['ParentCustomer_CD'])
    cust_cons = set(merged_df_clustered[merged_df_clustered['Item_TX'].isin(cons)]['ParentCustomer_CD'])
    diff = list(cust_ant - cust_cons)

    general_records_2.append({
        'antecedent'      : ', '.join(rule.lhs),
        'consequent'      : ', '.join(rule.rhs),
        'support'         : rule.support,
        'confidence'      : rule.confidence,
        'lift'            : rule.lift,
        'cust_bought_ant' : len(cust_ant),
        'raw_diff_list'   : diff,
        'clean_diff_list' : clean_ids(diff),
        'diff_count'      : len(clean_ids(diff))
    })

general_next15 = pd.DataFrame(general_records_2)

# Build ParentCustomer_CD - ParentCustomer_TX 
cust_map = (
    merged_df_clustered[['ParentCustomer_CD','ParentCustomer_TX']]
    .drop_duplicates()
    .set_index('ParentCustomer_CD')['ParentCustomer_TX']
    .to_dict()
)

# IDs  -names 
for df in (general_top15, general_next15):
    df['Missing_ParentCustomer_TXs'] = df['clean_diff_list'].apply(
        lambda ids: [cust_map.get(i, f"ID:{i}") for i in ids]
    )
    df['Missing_ParentCustomer_TXs_str'] = df['Missing_ParentCustomer_TXs'].apply(lambda L: "; ".join(L))

# Export
out_path = "G:/Mi unidad/Antwerp/Data engineering/Solved/Market basket/general_mba_support_top30_cleaned_with_names.xlsx"
with pd.ExcelWriter(out_path, engine='openpyxl') as writer:
    general_top15 .to_excel(writer, sheet_name='General_Top15', index=False)
    general_next15.to_excel(writer, sheet_name='General_Next15', index=False)
print("Exported with customer-names to:", out_path)

for df, title in [(general_top15, "Top 15"), (general_next15, "Next 15")]:
    pdf = df[df['diff_count'] > 0]
    plt.figure(figsize=(8, max(4, len(pdf)*0.5)), tight_layout=True)
    sns.barplot(
        data=pdf,
        y='antecedent',
        x='diff_count',
        hue='consequent',
        dodge=False
    )
    plt.title(f" Overall: # Customers Bought Antecedent but NOT Consequent\n({title} Rules by Support)")
    plt.xlabel("Number of Customers Missing Consequent")
    plt.ylabel("Antecedent Item")
    plt.legend(title="Consequent", bbox_to_anchor=(1.05,1), loc='upper left')
    plt.show()


In [1]:
git add "Product_portfolio _patterns.ipynb"
git commit -m "Agrega notebook de patrones de portafolio"

SyntaxError: invalid syntax (323616000.py, line 1)